In [2]:
import pandas as pd
import matplotlib.pyplot as plt

# 中央区、港区、千代田区のCSVファイルを読み込む
chuo_data = pd.read_csv("/Users/nishizawarharuto/DSprogFinalProject/chuo_properties_no_built_year.csv")
minato_data = pd.read_csv("/Users/nishizawarharuto/DSprogFinalProject/minato_properties_no_built_year.csv")
chiyoda_data = pd.read_csv("/Users/nishizawarharuto/DSprogFinalProject/chiyoda_properties_no_built_year.csv")

# 全てのデータを統合
all_data = pd.concat([chuo_data, minato_data, chiyoda_data], ignore_index=True)

# データの確認
print("[INFO] データフレームの基本情報")
print(all_data.info())
print(all_data.head())

[INFO] データフレームの基本情報
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9848 entries, 0 to 9847
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   物件名     9848 non-null   object
 1   販売価格    9848 non-null   object
 2   所在地     9848 non-null   object
 3   専有面積    9848 non-null   object
 4   間取り     9848 non-null   object
dtypes: object(5)
memory usage: 384.8+ KB
None
            物件名    販売価格               所在地                 専有面積       間取り
0       アペニンハイム  3380万円   東京都中央区勝どき３-4-18  35.91m2（10.86坪）（登記）      1LDK
1     浜町グランドハイツ  3990万円       東京都中央区日本橋中洲             36m2（壁芯）  1K+S（納戸）
2     浜町グランドハイツ  4180万円       東京都中央区日本橋中洲     36m2（10.88坪）（壁芯）  1K+S（納戸）
3  ワコー東日本橋マンション  4250万円      東京都中央区日本橋浜町１  42.74m2（12.92坪）（壁芯）      1LDK
4    箱崎ダイヤマンション  4480万円  東京都中央区日本橋箱崎町13-8  43.82m2（13.25坪）（壁芯）      1LDK


In [5]:
# 価格を数値に変換（万円単位に統一）
def convert_price(price):
    try:
        # "億"と"万円"を同時に含む場合
        if "億" in price and "万円" in price:
            parts = price.split("億")
            oku = float(parts[0]) * 10000  # 億を万円に変換
            man = float(parts[1].replace("万円", "").strip())  # 万円を数値化
            return oku + man
        # "億円"のみの場合
        elif "億" in price:
            return float(price.replace("億円", "").strip()) * 10000
        # "万円"のみの場合
        elif "万円" in price:
            return float(price.replace("万円", "").strip())
        else:
            return None  # 不正なデータの場合
    except Exception as e:
        print(f"[WARNING] 価格の変換に失敗しました: {price}, エラー: {e}")
        return None

# 販売価格列を変換
all_data['販売価格'] = all_data['販売価格'].apply(convert_price)

# 面積を数値に変換（m²部分を削除）
all_data['専有面積'] = all_data['専有面積'].str.extract(r'(\d+\.\d+)').astype(float)

# データの確認
print(all_data.head())

            物件名    販売価格               所在地   専有面積       間取り
0       アペニンハイム  3380.0   東京都中央区勝どき３-4-18  35.91      1LDK
1     浜町グランドハイツ  3990.0       東京都中央区日本橋中洲    NaN  1K+S（納戸）
2     浜町グランドハイツ  4180.0       東京都中央区日本橋中洲  10.88  1K+S（納戸）
3  ワコー東日本橋マンション  4250.0      東京都中央区日本橋浜町１  42.74      1LDK
4    箱崎ダイヤマンション  4480.0  東京都中央区日本橋箱崎町13-8  43.82      1LDK
